In [1]:
from data.data import get_data
from visualize.graphs_eval import stock_line_graph
import pandas as pd

In [2]:
ticker = "GOOGL"
dates = ["1990-01-01", "2022-11-20"]
data = get_data(ticker=ticker, dates=dates)
data = data.sort_index(ascending=True)
data.head()


GOOGL data acquired


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2004-08-19,2.604104,2.401401,2.502503,2.511011,893181924.0,2.511011
2004-08-20,2.729730,2.515015,2.527778,2.710460,456686856.0,2.710460
2004-08-21,2.729730,2.515015,2.527778,2.710460,456686856.0,2.710460
2004-08-22,2.729730,2.515015,2.527778,2.710460,456686856.0,2.710460
2004-08-23,2.839840,2.728979,2.771522,2.737738,365122512.0,2.737738


In [17]:
data.to_csv("data/data.csv")

In [3]:
stock_line_graph(df=data, ticker=ticker)

In [4]:
data.shape

(6666, 6)

# Model LSTM

In [5]:
# import numpy as np
# import torch
# import torch
# import torch.nn as nn
# import torch.functional as F
# from torch.nn import LSTM
# import torch.optim as optim
# from torch.autograd import Variable
# import blankly

# def  episode_gen(data: np.ndarray, seq_length: int, output_size:int) -> tuple[np.ndarray, np.ndarray]:
#     x = []
#     y = []
#     # Loop through data, adding input data to x array and output data to y array
#     for i in  range(len(data)-seq_length):
#         _x = data[i:(i+seq_length - output_size)]
#         _y = data[i+seq_length - output_size:i + seq_length]
#         x.append(_x)
#         y.append(_y)

#     return np.array(x),np.array(y)

# episode_gen(data=data.Close.values, seq_length=8, output_size=3)
# def init_NN(state: blankly.StrategyState):

#     interface = state.interface
#     resolution = state.resolution
#     variables = state.variables

#     # Get price data

#     # variables['history'] = interface.history(symbol, 300, resolution, return_as='list')['close']

#     '''We use Blankly's built-in indicator functions to calculate indicators we can use along with price data to predict future prices
#     '''
#     rsi = blankly.indicators.rsi(state.variables['history'])
#     macd = blankly.indicators.macd(state.variables['history'])

#     '''We'll break the historical Ethereum data into 8 day episodes

#     and attempt to predict the final three days using the first 5.

#     '''
#     seq_length = 8
#     output_size = 3

#     '''Feature engineering -- here, we calculate the price change from the day before
#     as a ratio. This is useful because it means we have less issues with scaling with the model,
#     as the data will all already be in roughly the same range. We ignore the first 25 elements
#     because we want every observation to have corresponding RSI + MACD data, and MACD requires
#     26 periods.
#     '''

#     x = [variables['history'][i] / variables['history'][i-1] for i in  range(25,len(variables['history']))]
#     x, y = episode_gen(x, seq_length, output_size)
#     y = Variable(torch.Tensor(np.array(y))).unsqueeze(0)

#     #RSI data gathering
#     x_rsi = rsi[11:]
#     x_rsi,_ = episode_gen(x_rsi,seq_length, output_size)

#     #MACD data gathering
#     macd_vals,_ = episode_gen(macd[0], seq_length, output_size)
#     macd_signals,_ = episode_gen(macd[1],seq_length, output_size)

#     '''In this section, we put all the features we just extracted into one NumPy array
#     which we then convert to a PyTorch tensor that we can run our model on.

#     '''

#     x_agg = np.zeros((len(macd_signals),seq_length-output_size, 4))

#     for i in  range(len(macd_signals)):
#         for j in  range(seq_length - output_size):
#             x_agg[i][j][0] = x[i][j]
#             x_agg[i][j][1] = x_rsi[i][j]
#             x_agg[i][j][2] = macd_vals[i][j]
#             x_agg[i][j][3] = macd_signals[i][j]
#     x_tot = Variable(torch.Tensor(x_agg))
# x_tot = init_NN()

# num_epochs = 10000
# learning_rate = 0.0003

# lstm = LSTM(4,20, batch_first = True)
# lin = nn.Linear(20,3)

# criterion = torch.nn.MSELoss() # mean-squared error for regression

# #Optimizer: make sure to include parameters of both linear layer and LSTM

# optimizer = torch.optim.Adam([
# {'params': lstm.parameters()},
# {'params': lin.parameters()}
# ], lr=learning_rate)

# # Train the model
# for epoch in  range(num_epochs):

#     #run model
#     outputs, (h_n, c_n) = lstm(x_tot)
#     out = lin(h_n)
#     out = 0.5 + F.sigmoid(out)
#     optimizer.zero_grad()

#     loss = criterion(out, y) #calculate loss function
#     loss.backward() #backprop
#     optimizer.step() #gradient descent



#     #Output loss functions every 500 epochs so we can make sure the model is training

#     if epoch % 500 == 0:
#         print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))



# '''We use this in the trading algorithm for more stability.

# Essentially, instead of relying on a single output of the model

# to tell us whether to buy or sell, we average the readings from three different calculations

# (3 days before, 2 days before, day before)

# '''
# lastthree = [[0,0],[0,0],[0,0]]

In [6]:
data.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2004-08-19,2.604104,2.401401,2.502503,2.511011,893181924.0,2.511011
2004-08-20,2.729730,2.515015,2.527778,2.710460,456686856.0,2.710460
2004-08-21,2.729730,2.515015,2.527778,2.710460,456686856.0,2.710460
2004-08-22,2.729730,2.515015,2.527778,2.710460,456686856.0,2.710460
2004-08-23,2.839840,2.728979,2.771522,2.737738,365122512.0,2.737738


In [7]:
# Mid prices calculation
high_prices = data.loc[:, "High"].values
low_prices = data.loc[:, "Low"].values
mid_prices = (high_prices+low_prices)/2

In [8]:
print(high_prices)
print(low_prices)
print(mid_prices)

[ 2.60410404  2.72972989  2.72972989 ... 99.63999939 99.27999878
 98.90000153]
[ 2.40140104  2.51501489  2.51501489 ... 97.63999939 96.79000092
 96.37000275]
[ 2.50275254  2.62237239  2.62237239 ... 98.63999939 98.03499985
 97.63500214]


In [9]:
data_size = len(mid_prices)

In [10]:
# Split data, train/test split 80%/20%
train_percentage = 0.8
data_split = int(data_size*train_percentage)
train_data = mid_prices[:data_split]
test_data = mid_prices[data_split:]

In [11]:
len(test_data)

1334

In [12]:
from sklearn.preprocessing import MinMaxScaler

In [14]:
# Scale the data to be between 0 and 1
# When scaling remember! You normalize both test and train data with respect to training data
# Because you are not supposed to have access to test data
scaler = MinMaxScaler()
train_data = train_data.reshape(-1,1)
test_data = test_data.reshape(-1,1)


In [15]:
# Train the Scaler with training data and smooth data
smoothing_window_size = 2500
for di in range(0,10000,smoothing_window_size):
    scaler.fit(train_data[di:di+smoothing_window_size,:])
    train_data[di:di+smoothing_window_size,:] = scaler.transform(train_data[di:di+smoothing_window_size,:])

# You normalize the last bit of remaining data
scaler.fit(train_data[di+smoothing_window_size:,:])
train_data[di+smoothing_window_size:,:] = scaler.transform(train_data[di+smoothing_window_size:,:])


ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by MinMaxScaler.

In [1]:
import torch
import torch.nn as nn


In [ ]:
class Time2Vec(nn.Module):
    def __init__(self, seq_len) -> None:
        super(Time2Vec, self).__init__()
        self.seq_len = seq_len
        self.weights_linear = nn.Parameter(torch.empty(self.seq_len), requires_grad=True)
        nn.init.uniform_(self.weights_linear)

        self.bias_linear = nn.Parameter(torch.empty(self.seq_len), requires_grad=True)
        nn.init.uniform_(self.bias_linear)

        self.weights_periodic = nn.Parameter(torch.empty(self.seq_len), requires_grad=True)
        nn.init.uniform_(self.weights_periodic)

        self.bias_periodic = nn.Parameter(torch.empty(self.seq_len), requires_grad=True)
        nn.init.uniform_(self.bias_periodic)

    def forward(self, x):
        x = torch.mean(x[:,:,:4], dim=-1)
        time_linear = self.weights_linear * x + self.bias_linear
        time_linear = time_linear.unsqueeze(-1)
        time_periodic = torch.sin(torch.multiply(x, self.weights_periodic) + self.bias_periodic)
        time_periodic = time_periodic.unsqueeze(-1)

        return torch.cat([time_linear, time_periodic], dim=-1)
